In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=0.8,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
    {
        "movie": "Dune: Part Two",
        "answer": """
Director: Denis Villeneuve
Actors: Timothée Chalamet, Zendaya, etc.
Budget: $190,000,000
Profit: $711,844,358
Genres: Action, Sci-Fi, etc.
Synopsis: Paul Atreides unites with Chani and the Fremen while on a warpath of revenge against the conspirators who destroyed his family. Facing a choice between the love of his life and the fate of the known universe, he endeavors to prevent a terrible future only he can foresee.
""",
    },
    {
        "movie": "Spider-Man: Across the Spider-Verse",
        "answer": """
Director: Joaquim Dos Santos
Actors: Shameik Moore, Hailee Steinfeld, etc.
Budget: $150,000,000
Profit: $690,001,091
Genres: Adventure, Animation, etc.
Synopsis: Miles Morales returns for the next chapter of the Oscar®-winning Spider-Verse saga, an epic adventure that will transport Brooklyn's full-time, friendly neighborhood Spider-Man across the Multiverse to join forces with Gwen Stacy and a new team of Spider-People to face off with a villain more powerful than anything they have ever encountered.
""",
    },
    {
        "movie": "The Dark Knight",
        "answer": """
Director: Christopher Nolan
Actors: Christian Bale, Heath Ledger, Sir Michael Caine, etc
Budget: $185,000,000
Profit: $1,007,336,937
Genres: Crime, Thriller, etc.
Synopsis: Set within a year after the events of Batman Begins (2005), Batman, Lieutenant James Gordon, and new District Attorney Harvey Dent successfully begin to round up the criminals that plague Gotham City, until a mysterious and sadistic criminal mastermind known only as The Joker appears in Gotham, creating a new wave of chaos. Batman's struggle against The Joker becomes deeply personal, forcing him to confront everything he believes and improve his technology to stop him. A love triangle develops between Bruce Wayne, Dent, and Rachel Dawes.
""",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie lover. You give an information about the movie. When answering the 'Actor' part, you will answer maximum of 3 actors or less. When answering the 'Genres' part, you will answer maximum of 3 genres or less.",
        ),
        chat_prompt,
        (
            "human",
            "I want to know about the director, actors, budget, profit, genres, and the synopsis of the movie {movie}.",
        ),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "Avengers: Endgame"})

Director: Anthony Russo, Joe Russo
Actors: Robert Downey Jr., Chris Evans, Scarlett Johansson
Budget: $356,000,000
Profit: $2,798,000,000
Genres: Action, Adventure, Sci-Fi
Synopsis: After the devastating events of Avengers: Infinity War (2018), the universe is in ruins. With the help of remaining allies, the Avengers assemble once more in order to undo Thanos' actions and restore order to the universe.

AIMessageChunk(content="Director: Anthony Russo, Joe Russo\nActors: Robert Downey Jr., Chris Evans, Scarlett Johansson\nBudget: $356,000,000\nProfit: $2,798,000,000\nGenres: Action, Adventure, Sci-Fi\nSynopsis: After the devastating events of Avengers: Infinity War (2018), the universe is in ruins. With the help of remaining allies, the Avengers assemble once more in order to undo Thanos' actions and restore order to the universe.")